In [1]:
import tensorflow as tf
import numpy as np

tf.set_random_seed(777) # for reproducibility

x_data = np.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=np.float32)
y_data = np.array([[0], [1], [1], [0]], dtype=np.float32)

X = tf.placeholder(tf.float32, [None, 2], name='x')
Y = tf.placeholder(tf.float32, [None, 1], name='y')

# 좀 더 구조적으로 tensor에 이름 지어주는 방법
with tf.name_scope('Layer1'):
    W1 = tf.Variable(tf.random_normal([2, 2]), name='weight_1')
    b1 = tf.Variable(tf.random_normal([2]), name='bias_1')
    layer1 = tf.sigmoid(tf.matmul(X, W1) + b1)
    
    tf.summary.histogram('W1', W1)
    tf.summary.histogram('b1', b1)
    tf.summary.histogram('Layer1', layer1)
    
with tf.name_scope('Layer2'):
    W2 = tf.Variable(tf.random_normal([2, 1]), name='weight_2')
    b2 = tf.Variable(tf.random_normal([1]), name='bias_2')
    hypothesis = tf.sigmoid(tf.matmul(layer1, W2) + b2)
    
    tf.summary.histogram('W2', W2)
    tf.summary.histogram('b2', b2)
    tf.summary.histogram('Hypothesis', hypothesis)
    
with tf.name_scope('Cost'):
    cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1-Y) * tf.log(1-hypothesis))
    tf.summary.scalar('Cost', cost)
    
with tf.name_scope('Train'):
    train = tf.train.AdamOptimizer(learning_rate = 0.01).minimize(cost)
    
# Accuracy
predicted = tf.cast(hypothesis>0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))
tf.summary.scalar('accuracy', accuracy)

with tf.Session() as sess:
    # tensorboard --logdir=./logs/xor_logs
    merged_summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter('./logs/xor_logs_r0_01')
    writer.add_graph(sess.graph)
    
    sess.run(tf.global_variables_initializer())
    
    for step in range(10001):
        _, summary, cost_val = sess.run(
            [train, merged_summary, cost], feed_dict={X:x_data, Y:y_data}
        )
        writer.add_summary(summary, global_step=step)
        
        if step % 1000 == 0:
            print(step, cost_val)
            
    h, p, a = sess.run(
        [hypothesis, predicted, accuracy], feed_dict={X:x_data, Y:y_data}
    )
    
    print(f'\nHypothesis:\n{h} \nPredicted:\n{p} \nAccuracy:\n{a}')
    

C:\Users\ui88g\anaconda3\envs\tf113\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\ui88g\anaconda3\envs\tf113\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\ui88g\anaconda3\envs\tf113\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\ui88g\anaconda3\envs\tf113\lib\site-packages\tensorflow\python\f

Instructions for updating:
Colocations handled automatically by placer.
0 0.7168676
1000 0.022373727
2000 0.0063154213
3000 0.0027729843
4000 0.0014256047
5000 0.00078983756
6000 0.00045537972
7000 0.00026846584
8000 0.00016036436
9000 9.6549324e-05
10000 5.8384467e-05

Hypothesis:
[[6.1310493e-05]
 [9.9993694e-01]
 [9.9995077e-01]
 [5.9751532e-05]] 
Predicted:
[[0.]
 [1.]
 [1.]
 [0.]] 
Accuracy:
1.0


In [2]:
!tensorboard --logdir=./logs/xor_logs

^C
